In [48]:
from importlib import reload 
import data_scrub
reload(data_scrub)
from data_scrub import *

pd.set_option('display.max_columns', None)

10 1055 1013 828 1308 1210 5424


In [ ]:
df = pd.read_csv('data_raw/otodom_last7_2024_04_18.csv')

df.reset_index(drop=True)
df['furnishings_list'] = df['furnishings'].str.replace("'", "").str.split(', ')
exploded_df = df['furnishings_list'].explode()
dummies = pd.get_dummies(exploded_df)
dummies = dummies.groupby(level=0).max()#.astype(bool)
translations = {
    'kuchenka': 'stove',
    'lodówka': 'fridge',
    'meble': 'furniture',
    'piekarnik': 'oven',
    'pralka': 'washing_machine',
    'telewizor': 'television',
    'zmywarka': 'dishwasher'
}
dummies.rename(columns=translations, inplace=True)

df = df.join(dummies)
df[['furnishings', 'stove', 'fridge', 'furniture', 'oven', 'washing_machine', 'television', 'dishwasher']]

NameError: name 'pd' is not defined

In [ ]:
df[df.stove.eq(False)].reset_index().link[0]

In [3]:
subways_location = {
    'Plac Wilsona': {'Latitude': 52.26902, 'Longitude': 20.985},
    'Wilanowska': {'Latitude': 52.18098, 'Longitude': 21.02364},
    'Centrum': {'Latitude': 52.23055, 'Longitude': 21.01091},
    'Stokłosy': {'Latitude': 52.15583, 'Longitude': 21.0347},
    'Wierzbno': {'Latitude': 52.18932, 'Longitude': 21.01726},
    'Pole Mokotowskie': {'Latitude': 52.2079, 'Longitude': 21.00837},
    'Swiętokrzyska': {'Latitude': 52.23499, 'Longitude': 21.00835},
    'Ratusz': {'Latitude': 52.24476, 'Longitude': 21.00082},
    'Marymont': {'Latitude': 52.27178, 'Longitude': 20.97232},
    'Imielin': {'Latitude': 52.14952, 'Longitude': 21.04543},
    'Kabaty': {'Latitude': 52.13109, 'Longitude': 21.06604},
    'Ursynów': {'Latitude': 52.16217, 'Longitude': 21.02858},
    'Natolin': {'Latitude': 52.14033, 'Longitude': 21.05776},
    'Bemowo': {'Latitude': 52.2393879, 'Longitude': 20.9175868},
    'Ulrychów': {'Latitude': 52.2402211, 'Longitude': 20.9286984},
    'Księcia Janusza': {'Latitude': 52.238989, 'Longitude': 20.9413589},
    'Młynów': {'Latitude': 52.2370368, 'Longitude': 20.9582514},
    'Płocka': {'Latitude': 52.2331322, 'Longitude': 20.9672229},
    'Rondo Daszyńskiego': {'Latitude': 52.2302503, 'Longitude': 20.9812049},
    'Rondo ONZ': zzz{'Latitude': 52.2326673, 'Longitude': 20.9943419},
    'Nowy Świat Uniwersytet': {'Latitude': 52.2372522, 'Longitude': 21.0163951},
    'Centrum Nauki Kopernik': {'Latitude': 52.2405898, 'Longitude': 21.029495},
    'Stadion Narodowy': {'Latitude': 52.2468026, 'Longitude': 21.0437104},
    'Wileński': {'Latitude': 52.2552165, 'Longitude': 21.0373375},
    'Szwedzka': {'Latitude': 52.2658294, 'Longitude': 21.0477777},
    'Targówek+Mieszkaniowy': {'Latitude': 52.278611, 'Longitude': 21.0478345},
    'Zacisze': {'Latitude': 52.278611, 'Longitude': 21.0478345},
    'Kondratowicza': {'Latitude': 52.2844033, 'Longitude': 21.0489269},
    'Bródno': {'Latitude': 52.2844033, 'Longitude': 21.0489269}}

In [62]:
df_1 = pd.read_csv('data_raw/otodom_last7_2024_03_22.csv')

In [ ]:
import morfeusz2

morf = morfeusz2.Morfeusz()

# Expanded furnishing keywords
keywords = ['zmywarka']

def contains_keywords_morf(description: str, keywords: list):
    doc = nlp(description.lower())
    contains_keywords = False
    for token in doc:
        analysis = morf.analyse(token.text)
        contains_keywords = True if analysis[0][2][1] in keywords else contains_keywords
        if contains_keywords:
            break
            
    return contains_keywords

contains_keywords_morf("Po obiedzie włożyłem brudne naczynia do zmywarką, aby mieć więcej czasu na relaks.")

In [ ]:
furnishing_keywords = [
    'umeblować', 'wyposażyć', 'mebel', 'łóżko', 'sofa', 'szafa', 'pralka'
]

In [ ]:
analysis = morf.analyse("klimatyzacją")
furnished = False
for entry in analysis:
    print(entry[2][1])

In [ ]:
import morfeusz2

morf = morfeusz2.Morfeusz()

keywords = ['umeblować', 'wyposażyć', 'mebel', 'łóżko', 'sofa', 'szafa', 'pralka']

analysis = morf.analyse("klimatyzacja")
furnished = False
for entry in analysis:
    furnished = True if entry[2][1] in furnishing_keywords else furnished
    
print(furnished)

In [ ]:
[]